In [ ]:
#  UserWarningを省略したい場合はこのコードを実行すること
import warnings
warnings.filterwarnings("ignore", category=UserWarning)

In [ ]:
%%time
import pandas as pd
import geopandas as gpd
import os
import gc
import matplotlib.pyplot as plt
from matplotlib.colors import to_hex
from plotnine import *
plt.rcParams['font.family'] = "Noto Sans CJK JP"

kihon_list = [file for file in os.listdir("/workspace/polygon/") if file.endswith(".shp")]
for i in kihon_list:
    d = "/workspace/polygon/" + i
    polygon = gpd.read_file(d)
    polygon = polygon.to_crs(epsg=3857)
    polygon['PD'] = polygon['JINKO'] / (polygon['AREA'] / 1000000)
    t =  polygon['PREF'].iloc[1] + polygon['CITY'].iloc[1]+polygon['PREF_NAME'].iloc[1] + polygon['CITY_NAME'].iloc[1]
    title = polygon['PREF_NAME'].iloc[1] + polygon['CITY_NAME'].iloc[1]
    try:
        polygon['legend'] = pd.qcut(polygon['PD'],5)
    except ValueError as e:
        error_path = "/workspace/result/choro/" + t + ".txt"
        with open(error_path, 'w') as f:
            f.write('creating choro is failed:\n')
            f.write(f'ValueError:{str(e)}')
    else:
        cmap = plt.get_cmap('jet', 5)
        cmap_colors = [to_hex(cmap(i / 5)) for i in range(5)]
        
        choro = (ggplot() +
                theme(text=element_text(family='Noto Sans CJK JP'),
                      panel_background=element_rect(fill='white'),
                      plot_background=element_rect(fill='white'),
                      panel_grid_major=element_blank(),
                      panel_grid_minor=element_blank(),
                      axis_title=element_blank(),
                      axis_text=element_blank(),
                      axis_ticks=element_blank()) +
                geom_map(data=polygon, mapping=aes(fill='legend'), color= None) +
                scale_fill_manual(values=cmap_colors) +
                labs(fill='人口密度', title=title) + 
                coord_equal())
        file_path = "/workspace/result/choro/" + t + ".png"
        choro.save(file_path)
        del choro
        gc.collect()